In [1]:
#in the name of GOD

In [60]:
import pandas as pd 
import numpy as np
import pickle

from copy import deepcopy
import time
import six
import sys

from fastdtw import fastdtw
sys.modules['sklearn.externals.six'] = six
from id3 import Id3Estimator, export_text
from sklearn.metrics import accuracy_score, classification_report


In [3]:
#data_read
#prepare Dataset gyroscope
dfg=pd.read_csv('16\giroscopio_terra.csv')
dfg['ts']=pd.to_datetime(dfg.apply(lambda x:(round((x.uptimeNanos-dfg.uptimeNanos[0])/1000000)*1000000),axis=1))
dm=dfg['ts']
dfg=dfg.drop('uptimeNanos',1).drop('timestamp',1)
dfg=dfg.set_index('ts').rename(columns={'x': 'g-x','y': 'g-y','z': 'g-z'})
#prepare Dataset accelarator
dfa=pd.read_csv('16\\acelerometro_terra.csv')
dfa['ts']=dm
dfa=dfa.drop('uptimeNanos',1).drop('timestamp',1)
dfa=dfa.set_index('ts').rename(columns={'x': 'a-x','y': 'a-y','z': 'a-z'})
df1 = pd.concat([dfa, dfg], axis=1, join='outer')
#prepare Dataset gyroscope
dfg=pd.read_csv('17\giroscopio_terra.csv')
dfg['ts']=pd.to_datetime(dfg.apply(lambda x:(round((x.uptimeNanos-dfg.uptimeNanos[0])/1000000)*1000000),axis=1))
dm=dfg['ts']
dfg=dfg.drop('uptimeNanos',1).drop('timestamp',1)
dfg=dfg.set_index('ts').rename(columns={'x': 'g-x','y': 'g-y','z': 'g-z'})
#prepare Dataset accelarator
dfa=pd.read_csv('17\\acelerometro_terra.csv')
dfa['ts']=dm
dfa=dfa.drop('uptimeNanos',1).drop('timestamp',1)
dfa=dfa.set_index('ts').rename(columns={'x': 'a-x','y': 'a-y','z': 'a-z'})
df2 = pd.concat([dfa, dfg], axis=1, join='outer')
#prepare Dataset gyroscope
dfg=pd.read_csv('20\\giroscopio_terra.csv')
dfg['ts']=pd.to_datetime(dfg.apply(lambda x:(round((x.uptimeNanos-dfg.uptimeNanos[0])/1000000)*1000000),axis=1))
dm=dfg['ts']
dfg=dfg.drop('uptimeNanos',1).drop('timestamp',1)
dfg=dfg.set_index('ts').rename(columns={'x': 'g-x','y': 'g-y','z': 'g-z'})
#prepare Dataset accelarator
dfa=pd.read_csv('20\\acelerometro_terra.csv')
dfa['ts']=dm
dfa=dfa.drop('uptimeNanos',1).drop('timestamp',1)
dfa=dfa.set_index('ts').rename(columns={'x': 'a-x','y': 'a-y','z': 'a-z'})
df3 = pd.concat([dfa, dfg], axis=1, join='outer')
#prepare Dataset gyroscope
dfg=pd.read_csv('21\giroscopio_terra.csv')
dfg['ts']=pd.to_datetime(dfg.apply(lambda x:(round((x.uptimeNanos-dfg.uptimeNanos[0])/1000000)*1000000),axis=1))
dm=dfg['ts']
dfg=dfg.drop('uptimeNanos',1).drop('timestamp',1)
dfg=dfg.set_index('ts').rename(columns={'x': 'g-x','y': 'g-y','z': 'g-z'})
#prepare Dataset accelarator
dfa=pd.read_csv('21\\acelerometro_terra.csv')
dfa['ts']=dm
dfa=dfa.drop('uptimeNanos',1).drop('timestamp',1)
dfa=dfa.set_index('ts').rename(columns={'x': 'a-x','y': 'a-y','z': 'a-z'})
df4 = pd.concat([dfa, dfg], axis=1, join='outer')

In [4]:
#supervised
window={
        'curva_direita_agressiva':226,
        'curva_esquerda_agressiva':226,
        'aceleracao_agressiva':226,
        'troca_faixa_direita_agressiva':126,
        'troca_faixa_esquerda_agressiva':126,
        'freada_agressiva':186,
       }

In [5]:
def TempEvent_genarator(df,L):
    M=round(L/4)
    temp_event=list()
    pointer=0
    stop=False
    #go on timeserise for event extraction
    while stop!=True:
        pointer=pointer + M
        l_min=pointer-int(L/2)
        l_max=pointer+int(L/2)
        if l_max<0:
            l_max=L/2
        if (l_min<0):
            l_min=0
        if (l_max>len(df)):
            l_max=len(df)
            stop=True
        temp_event.append(df[l_min:l_max])
    return temp_event
temp_event=TempEvent_genarator(df1,window['curva_direita_agressiva'])

In [6]:
with open("Model.txt", "rb") as fp:
    Model = pickle.load(fp)
with open("Report.txt", "rb") as fp:
    Report_of_Acc = pickle.load(fp)
with open("Model_param.txt", "rb") as fp:
    Model_param = pickle.load(fp)

In [51]:
def Harsh_Turn_Right(input_df,Model_param):
    r1=4
    x1=fastdtw(input_df,Model_param[r1][1])[0]
    if x1<=Model_param[r1][0]:
        return True
    else:
        r2=1
        x2=fastdtw(input_df,Model_param[r2][1])[0]
        if x2<=Model_param[r2][0]:
            r3=5
            x3=fastdtw(input_df,Model_param[r3][1])[0]
            if x3<=Model_param[r3][0]:
                return True
            else:
                return False
        else:
            r3=6
            x3=fastdtw(input_df,Model_param[r3][1])[0]
            if x3<=Model_param[r3][0]:
                return True
            else:
                return False
print('=========================')
print(' curva_direita_agressiva')
print('=========================')
print(Report_of_Acc['curva_direita_agressiva'])
print(export_text(Model['curva_direita_agressiva'].tree_))

Mdlprm=Model_param['curva_direita_agressiva']
print(Harsh_Turn_Right(temp_event[10],Mdlprm))


 curva_direita_agressiva
              precision    recall  f1-score   support

       False       0.99      1.00      0.99      3265
        True       0.96      0.59      0.73       120

    accuracy                           0.98      3385
   macro avg       0.97      0.80      0.86      3385
weighted avg       0.98      0.98      0.98      3385


4 <=913.94: True (37) 
4 >913.94
|   1 <=1353.63
|   |   5 <=1048.73: True (3) 
|   |   5 >1048.73: False (2253/25) 
|   1 >1353.63
|   |   6 <=1237.27: True (12) 
|   |   6 >1237.27: False (33/6) 

False


In [52]:
def Harsh_Turn_Left(input_df,Model_param):
    r1=4
    x1=fastdtw(input_df,Model_param[r1][1])[0]
    if x1<=Model_param[r1][0]:
        return True
    else:
        r2=5
        x2=fastdtw(input_df,Model_param[r2][1])[0]
        if x2<=Model_param[r2][0]:
            r3=9
            x3=fastdtw(input_df,Model_param[r3][1])[0]
            if x3<=Model_param[r3][0]:
                return True
            else:
                return False
        else:
            r3=8
            x3=fastdtw(input_df,Model_param[r3][1])[0]
            if x3<=Model_param[r3][0]:
                return True
            else:
                return False
print('=========================')
print(' curva_esquerda_agressiva')
print('=========================')
print(Report_of_Acc['curva_esquerda_agressiva'])
print(export_text(Model['curva_esquerda_agressiva'].tree_))

Mdlprm=Model_param['curva_esquerda_agressiva']
print(Harsh_Turn_Left(temp_event[10],Mdlprm))


 curva_esquerda_agressiva
              precision    recall  f1-score   support

       False       0.98      1.00      0.99      3262
        True       0.98      0.48      0.64       123

    accuracy                           0.98      3385
   macro avg       0.98      0.74      0.82      3385
weighted avg       0.98      0.98      0.98      3385


4 <=794.88: True (28) 
4 >794.88
|   5 <=1234.26
|   |   9 <=775.34: True (5) 
|   |   9 >775.34: False (2165/26) 
|   5 >1234.26
|   |   8 <=1377.82: True (11) 
|   |   8 >1377.82: False (117/17) 

False


In [54]:
def Harsh_acceleration(input_df,Model_param):
    r1=4
    x1=fastdtw(input_df,Model_param[r1][1])[0]
    if x1<=Model_param[r1][0]:
        return True
    else:
        r2=3
        x2=fastdtw(input_df,Model_param[r2][1])[0]
        if x2<=Model_param[r2][0]:
            return True
        else:
            r3=1
            x3=fastdtw(input_df,Model_param[r3][1])[0]
            if x3<=Model_param[r3][0]:
                return True
            else:
                return False
print('=========================')
print(' aceleracao_agressiva')
print('=========================')
print(Report_of_Acc['aceleracao_agressiva'])
print(export_text(Model['aceleracao_agressiva'].tree_))

Mdlprm=Model_param['aceleracao_agressiva']
print(Harsh_acceleration(temp_event[10],Mdlprm))


 aceleracao_agressiva
              precision    recall  f1-score   support

       False       0.91      1.00      0.95      1341
        True       0.95      0.22      0.36       174

    accuracy                           0.91      1515
   macro avg       0.93      0.61      0.65      1515
weighted avg       0.91      0.91      0.88      1515


4 <=466.58: True (13) 
4 >466.58
|   3 <=364.81: True (7) 
|   3 >364.81
|   |   1 <=341.85: True (6) 
|   |   1 >341.85: False (935/99) 

False


In [57]:
def Harsh_ChangeLine_Right(input_df,Model_param):
    r1=1
    x1=fastdtw(input_df,Model_param[r1][1])[0]
    if x1<=Model_param[r1][0]:
        return True
    else:
        r2=0
        x2=fastdtw(input_df,Model_param[r2][1])[0]
        if x2<=Model_param[r2][0]:
            r3=1
            x3=fastdtw(input_df,Model_param[r3][1])[0]
            if x3<=Model_param[r3][0]:
                return True
            else:
                return False
        else:
            return False
print('=========================')
print(' troca_faixa_direita_agressiva')
print('=========================')
print(Report_of_Acc['troca_faixa_direita_agressiva'])
print(export_text(Model['troca_faixa_direita_agressiva'].tree_))

Mdlprm=Model_param['troca_faixa_direita_agressiva']
print(Harsh_ChangeLine_Right(temp_event[10],Mdlprm))


 troca_faixa_direita_agressiva
              precision    recall  f1-score   support

       False       0.99      1.00      1.00      6481
        True       1.00      0.39      0.56        79

    accuracy                           0.99      6560
   macro avg       1.00      0.70      0.78      6560
weighted avg       0.99      0.99      0.99      6560


1 <=323.87: True (12) 
1 >323.87
|   0 <=490.01
|   |   1 <=585.89: False (2) 
|   |   1 >585.89: True (11) 
|   0 >490.01
|   |   3 <=607.13: False (4124/17) 
|   |   3 >607.13: False (410/16) 

False


In [63]:
def Harsh_ChangeLine_Left(input_df,Model_param):
    r1=3
    x1=fastdtw(input_df,Model_param[r1][1])[0]
    if x1<=Model_param[r1][0]:
        r2=1
        x2=fastdtw(input_df,Model_param[r2][1])[0]
        if x2<=Model_param[r2][0]:
            r3=0
            x3=fastdtw(input_df,Model_param[r3][1])[0]
            if x3<=Model_param[r3][0]:
                return True
            else:
                return False
        else:
            return True

    
    else:

        r2=1
        x2=fastdtw(input_df,Model_param[r2][1])[0]
        if x2<=Model_param[r2][0]:
            r3=1
            x3=fastdtw(input_df,Model_param[r3][1])[0]
            if x3<=Model_param[r3][0]:
                return True
            else:
                return False
        else:
            r3=2
            x3=fastdtw(input_df,Model_param[r3][1])[0]
            if x3<=Model_param[r3][0]:
                return True
            else:
                return False
print('=========================')
print(' troca_faixa_esquerda_agressiva')
print('=========================')
print(Report_of_Acc['troca_faixa_esquerda_agressiva'])
print(export_text(Model['troca_faixa_esquerda_agressiva'].tree_))

Mdlprm=Model_param['troca_faixa_esquerda_agressiva']
print(Harsh_ChangeLine_Left(temp_event[10],Mdlprm))


 troca_faixa_esquerda_agressiva
              precision    recall  f1-score   support

       False       0.99      1.00      1.00       805
        True       1.00      0.61      0.76        18

    accuracy                           0.99       823
   macro avg       1.00      0.81      0.88       823
weighted avg       0.99      0.99      0.99       823


3 <=743.73
|   1 <=511.02
|   |   0 <=443.77: False (1) 
|   |   0 >443.77: True (1) 
|   1 >511.02: True (5) 
3 >743.73
|   1 <=837.11
|   |   1 <=408.96: True (1) 
|   |   1 >408.96: False (548/6) 
|   1 >837.11
|   |   2 <=942.47: True (4) 
|   |   2 >942.47: False (10) 

False


In [64]:
def Harsh_Break(input_df,Model_param):
    r1=3
    x1=fastdtw(input_df,Model_param[r1][1])[0]
    if x1<=Model_param[r1][0]:
            return True
    else:
        r2=2
        x2=fastdtw(input_df,Model_param[r2][1])[0]
        if x2<=Model_param[r2][0]:
            return True
        else:
            r3=6
            x3=fastdtw(input_df,Model_param[r3][1])[0]
            if x3<=Model_param[r3][0]:
                return True
            else:
                return False
print('=========================')
print(' freada_agressiva')
print('=========================')
print(Report_of_Acc['freada_agressiva'])
print(export_text(Model['freada_agressiva'].tree_))

Mdlprm=Model_param['freada_agressiva']
print(Harsh_ChangeLine_Left(temp_event[10],Mdlprm))


 freada_agressiva
              precision    recall  f1-score   support

       False       0.96      1.00      0.98      2405
        True       0.93      0.45      0.61       182

    accuracy                           0.96      2587
   macro avg       0.95      0.72      0.79      2587
weighted avg       0.96      0.96      0.95      2587


3 <=527.94: True (23) 
3 >527.94
|   2 <=519.65: True (21) 
|   2 >519.65
|   |   6 <=750.43: True (17/4) 
|   |   6 >750.43: False (1682/63) 



KeyError: 1

In [ ]:
        '':126,
        '':186,
       